This notebook should be run as a Google Colab notebook

# Predict annotated trees in images

In [ ]:
from google.colab import drive
drive.mount('/content/drive')#, force_remount=True)
folder_path = '/content/drive/MyDrive/Thesis Johanne/Code'
import sys
sys.path.insert(0, folder_path)

Mounted at /content/drive


In [ ]:
# Install dependencies
!pip uninstall -y torchtext torchdata
!pip install torch==1.11.0+cu115 torchvision==0.12.0+cu115 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu115
!pip install -U openmim
!mim install mmengine
!mim install https://github.com/open-mmlab/mmdetection/archive/refs/tags/v2.28.2.zip

In [ ]:
import numpy as np
np.set_printoptions(formatter={'all':lambda x: str(x)})
import json
import mmcv
from mmdet.apis import inference_detector, init_detector, set_random_seed

/usr/local/lib/python3.10/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [ ]:
# Set random seed
seed = 3047
set_random_seed(seed, deterministic=False)

In [ ]:
def predict_annotations(model, filepath_gt, filepath_pred, filepath_imgs, ann_id):
    # Load ground truth annotations
    with open(filepath_gt, 'r') as fp:
        anns_gt = json.load(fp)
    n_imgs = len(anns_gt['images'])
    print(f'\tN. of images:', n_imgs)

    # Make predictions
    anns_pred = anns_gt.copy()
    anns_pred['annotations'] = []
    for i, img in enumerate(anns_gt['images']):
        im = mmcv.imread(f'{filepath_imgs}/{img["file_name"]}')
        preds = inference_detector(model, im)
        for p in preds[0]:
            x1, y1, x2, y2 = p[:-1]
            bbox = [x1, y1, x2-x1, y2-y1]
            bbox = [float(elem) for elem in bbox]
            anns_pred['annotations'].append({'id': ann_id,
                                            'image_id': img['id'],
                                            'category_id': 0,
                                            'bbox': bbox,
                                            'score': float(p[-1])})
            ann_id += 1

        # Print progress
        if (i > 0) & (i % 250 == 0):
            print(f'\t\t{round((i / n_imgs) * 100)}%')

    # Save predicted annotations
    with open(filepath_pred, 'w') as fp:
        json.dump(anns_pred, fp)

    return ann_id

In [ ]:
# Initialize the model
config = mmcv.Config.fromfile(f'{folder_path}/config.py')
checkpoint = f'{folder_path}/models/model.pth'
model = init_detector(config, checkpoint, device='cuda:0');

### Make predictions on Copenhagen dataset

In [ ]:
# Make predictions for each split
filepath_imgs = f'{folder_path}/data/streetviews'
ann_id = 0
for split in ['train', 'val', 'test']:
    print(f'{split.capitalize()}:')
    filepath_gt = f'{folder_path}/data/annotations/annotations_{split}_single_label.json'
    filepath_pred = f'{folder_path}/data/predictions/predictions_annotations_{split}.json'
    ann_id = predict_annotations(model, filepath_gt, filepath_pred, filepath_imgs, ann_id)

### Make predictions on Pasadena dataset

In [ ]:
# Make predictions for Pasadena dataset using original model
split = 'test'
print(f'{split.capitalize()}:')
filepath_imgs = f'{folder_path}/data/images/pasadena_2'
filepath_gt = f'{folder_path}/data/annotations/annotations_{split}_single_label_pd.json'
filepath_pred = f'{folder_path}/data/predictions/predictions_annotations_{split}_pd.json'
_ = predict_annotations(model, filepath_gt, filepath_pred, filepath_imgs, ann_id=0)

In [ ]:
# Initialize the finetuned model
config = mmcv.Config.fromfile(f'{folder_path}/config.py')
checkpoint = f'{folder_path}/models/model_finetuned.pth'
model = init_detector(config, checkpoint, device='cuda:0');

In [ ]:
# Make predictions on Pasadena dataset using fine-tuned model
split = 'test'
print(f'{split.capitalize()}:')
filepath_imgs = f'{folder_path}/data/images/pasadena_2'
filepath_gt = f'{folder_path}/data/annotations/annotations_{split}_single_label_pd.json'
filepath_pred = f'{folder_path}/data/predictions/predictions_annotations_{split}_pd_ft.json'
_ = predict_annotations(model, filepath_gt, filepath_pred, filepath_imgs, ann_id=0)